# Betting on NBA

In [ ]:
#TO IMPROVE
# min à 0 lorsque joue pas
# Prendre en compte blessure top7 joueurs - flag nb match avec sans gros joueur
# Poids sur les derniers matchs
# https://www.espn.com/nba/injuries
# https://www.espn.com/nba/player/gamelog/_/id/3136195/karl-anthony-towns not recognise
# O'Neale  / Lonnie Walker IV  / Robert Williams problem
# Probleme nom composé, avec des . ou deux prénoms double nom
# Prendre l'equipe en face en compte, nombre de points de rebonds, etc.
# 3 points tir 

#Prochaine etape : enregistrer cote unibet, pacakge uuid pour un identifiant


In [272]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4

In [326]:
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
import seaborn as sns
import urllib.request, json 
from IPython.core.display import display, HTML
from itertools import chain
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from datetime import datetime, date, timedelta
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
#driver = webdriver.Chrome("C:/Users/Valentin/Documents/Winamax/chromedriver")
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [328]:
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "nothing"
        

    get_page(Page)

###### Get list of Games
soup = BeautifulSoup(driver.page_source)

#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        print("") 

############ Parameters
get_page("https://www.espn.com/nba/schedule/_/date/" + date.today().strftime("%Y%m%d"))
soup = BeautifulSoup(driver.page_source)

list_Pages = []
list_Players = []
for teams in soup.find(class_='schedule has-team-logos align-left').findAll(class_='team-name'):
    p = "https://www.espn.com/nba/team/schedule/_/name/" + teams['href'].replace("/nba/team/_/name/", "").split("/")[0] + "/seasontype/2"
    list_Pages.append(p)
    list_Players.append(p.replace("/schedule/", "/stats/").replace("/seasontype/1", ""))

#Shorten the list
#list_Pages = list_Pages[0:1]
#list_Players = list_Players[0:1]

In [353]:
d = pd.DataFrame()
print("------------- " + str(int(len(list_Pages)/2)) + " games" + " ------------")

###### Get_Page function
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "nothing"
            
for jj in range(len(list_Pages)):
    Page = list_Pages[jj]
    Players = list_Players[jj]
    if(((jj+1)%2)!=0):
        sufx = str(int(jj/2.0+1)) + " "
    else:
        sufx = "  "
    print(sufx + Page.replace("https://www.espn.com/nba/team/schedule/_/name/", "").replace("/seasontype/2", ""))

    ###### Game/Games
    #get_page(special_match)
    get_page(Page)

    ###### Get list of Games
    soup = BeautifulSoup(driver.page_source)
    list_games = []

    for hrefs in soup.find(class_='Table__TBODY').findAll(class_='AnchorLink', href=re.compile("gameId")):
        list_games.append(hrefs['href'].replace("/game/", "/boxscore/"))

    index_games = soup.prettify().split("DATE\n")[1].split("data-idx")
    index_games = int(index_games[len(index_games)-1].split('">\n ')[0].split('="')[1])
    list_games = list_games[0:index_games-1]

    get_page(Players)
    soup = BeautifulSoup(driver.page_source)

    list_href_players = []
    for href_players in soup.find(class_='Table__TBODY').findAll(class_='AnchorLink', href=re.compile("player/_/id/")):
        list_href_players.append(href_players['href'].replace("https://www.espn.com/nba/player/_/id/", ""))

    def filterOut2(x):return x.split('-')[1]
    list_href_players = list(map(filterOut2, list_href_players))
    list_href_players

    for games in list_games:
        get_page(games)
        soup = BeautifulSoup(driver.page_source)

        ############ Creation of the dataset
        for i in range(0, 2): #0 to 1 because two teams
            my_table = soup.findAll(class_='content desktop')[i]
            Team_finded = my_table.find(class_='team-name')

            #Team_finded = my_table.contents[1]
            if 1 == 1:
                my_table = my_table.find(class_='mod-data')

                table_data = []
                for row in my_table.findAll('tr'):
                    row_data = []    
                    URLs = []    
                    
                    for u in row.findAll("a"):
                        URLs.append(u["href"].replace("/player/_/id/","/player/gamelog/_/id/"))
                        
                    for cell in row.findAll('td'):
                        row_data.append(cell.text)
                        
                    row_data = list(chain.from_iterable([row_data, URLs]))
                    
                    if(len(row_data) > 0):
                        if(any(string in row_data[0].lower() for string in list_href_players)):
                            #print(row_data)
                            if(row_data[0] == "TEAM" or row_data[0] == "" or len(row_data[0])==1):
                                do = "nothing"
                            else:
                                if('DNP' in row_data[1] or 
                                   row_data[0] == "TEAM"):
                                    data_item = {"Player": row_data[0],
                                                  "Minutes": 0}
                                else:
                                    if(jj<10):
                                        num = "0" + str(jj)
                                    else:
                                        num = str(jj)
                                    
                                    if float(row_data[14])>=20: PT20 = 1
                                    else: PT20 = 0
                                    if float(row_data[14])>=25: PT25 = 1
                                    else: PT25 = 0
                                    if float(row_data[14])>=30: PT30 = 1
                                    else: PT30 = 0
                                            
                                    data_item = {"Player": row_data[0],#.Substring(0, input.FirstIndexOf(".")),
                                                  "Minutes": row_data[1],
                                                  "FG": row_data[2],
                                                  "3PT": row_data[3],
                                                  "FT": row_data[4],
                                                  "OREB": row_data[5],
                                                  "DREB": row_data[6],
                                                  "REB": row_data[7],
                                                  "AST": row_data[8],
                                                 #calculated columns
                                                  "PTS_REB_AST": float(row_data[14])+float(row_data[7])+float(row_data[8]),
                                                  "PTS_REB": float(row_data[14])+float(row_data[7]),
                                                  "PTS_AST": float(row_data[14])+float(row_data[8]),
                                                  "REB_AST": float(row_data[7])+float(row_data[8]),
                                                  "20PTS": PT20,
                                                  "25PTS": PT25,
                                                  "30PTS": PT30,
                                                  "STL": row_data[9],
                                                  "BLK": row_data[10],
                                                  "TO": row_data[11],
                                                  "PF": row_data[12],
                                                  "PlusMinus": row_data[13],
                                                  "PTS": row_data[14],
                                                  "Team": num + "_" + Page.replace("https://www.espn.com/nba/team/schedule/_/name/", "").replace("/seasontype/2", ""),
                                                  "team":any(string in row_data[0].lower() for string in list_href_players),
                                                  "URL": row_data[15]
                                    }
                            table_data.append(data_item)

                data = pd.DataFrame(table_data)[:-1]
                d = pd.concat([d,data])
                
                
d.dropna(subset = ["REB"], inplace=True)
d["3PT"] = d["3PT"].astype(str)
d = d.assign(TPTW = d['3PT'].map(lambda x: x.split('-')[0]))
d["TPTW"] = d["TPTW"].astype(int, errors = 'raise')
d["REB"] = d["REB"].astype(int, errors = 'raise')
d["AST"] = d["AST"].astype(int, errors = 'raise')
d["PTS"] = d["PTS"].astype(int, errors = 'raise')
d["PTS_REB_AST"] = d["PTS_REB_AST"].astype(int, errors = 'raise')
d["PTS_REB"] = d["PTS_REB"].astype(int, errors = 'raise')
d["PTS_AST"] = d["PTS_AST"].astype(int, errors = 'raise')
d["Minutes"] = d["Minutes"].astype(int, errors = 'raise')

player_team = d[['Player', 'URL', 'Team']]
df = d.groupby(['Player', 'URL', 'Team']).describe(percentiles=[.5])
df = df[df['REB']['count']>=2]
a = df["Minutes"].sort_values(by = "mean", ascending = False, axis=0).index
df = df.reindex(a)
df.reset_index(level=1, inplace=True)
print("----------------------------------")

------------- 11 games ------------
1 bos
  wsh
2 orl
  det
3 ny
  no
4 tor
  ind
5 atl
  phi
6 utah
  chi
7 mia
  mem
8 sa
  mil
9 okc
  gs
10 den
  min
11 cle
  phx
----------------------------------


In [354]:
df["NbGames"]=df["Minutes"]["count"]
df["ScoreJ"]=df["PTS_REB_AST"]["mean"]
#df = df.sort_values('ScoreJ', ascending=False)
df = df[df.columns.drop(list(df.filter(regex='count')))]
df = df[df.columns.drop(list(df.filter(regex='std')))]
df = df[df.columns.drop(list(df.filter(regex='50%')))]
df = df[df.columns.drop(list(df.filter(regex='mean')))]
df = df[['URL', 'NbGames', 'PTS_REB_AST', 'PTS_REB', 'PTS_AST', 'REB_AST', 'PTS', 'REB', 'AST', 'TPTW', 'Minutes', '20PTS', '25PTS', '30PTS', 'ScoreJ']]

def make_clickable(val):
    if (isinstance(val, str) and val.find("http") != -1):
        return '<a href="{}">{}</a>'.format(val,val.split("id/")[1].split("/")[1])
    else:
        return val

cm = sns.light_palette("green", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
rd = sns.light_palette("red", as_cmap=True)
pr = sns.light_palette("purple", as_cmap=True)
rg = sns.light_palette("orange", as_cmap=True)
df = df.sort_values(by=["Team", "ScoreJ"], ascending=[True,False])
df = df[df['ScoreJ']>=6]
df = df.drop(['ScoreJ', 'Player'], 1)

df.reset_index(level=0, inplace=True)
df.reset_index(level=0, inplace=True)
df.style.format(make_clickable).background_gradient(cmap=rd, subset=['TPTW']).background_gradient(cmap=bl, subset=['PTS_REB_AST', 'PTS_REB', 'PTS_AST']).background_gradient(cmap=cm, subset=['PTS', 'REB', 'AST']).background_gradient(cmap=pr, subset=['Minutes']).background_gradient(cmap=rg, subset=['Minutes']).set_precision(0)


C:\Users\Valentin\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


IndexError: list index out of range

In [442]:
URL_prefix = "https://www.unibet.fr/zones/v3/sportnode/markets.json?nodeId=627956958&filter="

Spe_Joueurs = ["Sp%25C3%25A9cial%2520joueurs&marketname=Performance%2520du%2520Joueur%2520(Points%2520%252B%2520Rebonds%2520%252B%2520Passes)",  #Pts+Reb+Ast
               "Sp%25C3%25A9cial%2520joueurs&marketname=Performance%2520du%2520Joueur%2520(Points%2520%252B%2520Rebonds)", #Pts+Reb
               "Sp%25C3%25A9cial%2520joueurs&marketname=Performance%2520du%2520Joueur%2520(Points%2520%252B%2520Passes)",  #Pts+Ast
               "Sp%25C3%25A9cial%2520joueurs&marketname=Performance%2520du%2520Joueur%2520(Passes%2520%252B%2520Rebonds)", #Reb+Ast
               "Sp%25C3%25A9cial%2520joueurs&marketname=Nombre%2520de%2520rebonds%2520du%2520joueur", #Reb
               "Sp%25C3%25A9cial%2520joueurs&marketname=Nombre%2520de%2520passes%2520du%2520joueur", #Ast
               "Scoreurs&marketname=Joueur%2520marquant%252020%2520points%2520ou%2520plus", #20PTS
               "Scoreurs&marketname=Joueur%2520marquant%252025%2520points%2520ou%2520plus", #25PTS
               "Scoreurs&marketname=Joueur%2520marquant%252030%2520points%2520ou%2520plus", #30PTS
               "Scoreurs&marketname=Joueur%2520marquant%252035%2520points%2520ou%2520plus", #35PTS
               "Scoreurs&marketname=Joueur%2520marquant%252040%2520points%2520ou%2520plus"  #40PTS
              ]
Spe_Joueurs_indx = ["PTS_REB_AST", "PTS_REB"
                    , "PTS_AST", "REB_AST", "REB", "AST", "20PTS", "25PTS", "30PTS", "35PTS", "40PTS"
                   ]


df2 = pd.DataFrame()
for b in range(len(Spe_Joueurs_indx)):
    u = URL_prefix+Spe_Joueurs[b]
    with urllib.request.urlopen(u) as url:
        data = json.loads(url.read().decode())
        
    row_data = []  
    
    for d in data['marketsByType'][0]['days']:
        for e in d['events']:
            for j in e['markets'][0]['selections']:
                if(j['name'].find("Moins") == -1):
                    if(b<6): #from 20PTS
                        data_item = {"Player": j['name'].split(" - ")[0],
                                     Spe_Joueurs_indx[b]:[int(s) for s in j['name'].split(" - ")[1].replace(".","").replace(",","").split() if s.isdigit()][0]/10,
                                     "Odd+"+Spe_Joueurs_indx[b]:round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)
                                    }
                    else:
                        data_item = {"Player": j['name'],
                                     "Odd+"+Spe_Joueurs_indx[b]:round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)                         
                                    }
                    row_data.append(data_item)
    data = pd.DataFrame(row_data)
    if b == 0:
        df2 = data
    else:
        df2 = df2.merge(data, on='Player', how='left')
df2

,Player,PTS_REB_AST,Odd+PTS_REB_AST,PTS_REB,Odd+PTS_REB,PTS_AST,Odd+PTS_AST,REB_AST,Odd+REB_AST,REB,Odd+REB,AST,Odd+AST,Odd+20PTS,Odd+25PTS,Odd+30PTS,Odd+35PTS,Odd+40PTS
0,Aaron Holiday,9.5,1.78,7.5,1.80,7.5,1.76,4.5,1.96,1.5,1.58,1.5,1.48,8.00,NaN,NaN,NaN,NaN
1,Bradley Beal,37.5,1.76,33.5,1.78,31.5,1.78,10.5,1.92,5.5,1.60,4.5,2.15,1.17,1.46,2.15,3.50,5.40
2,Davis Bertans,10.5,1.76,10.5,1.84,8.5,1.80,2.5,1.64,2.5,2.15,0.5,1.92,6.00,NaN,NaN,NaN,NaN
3,Deni Avdija,11.5,1.76,10.5,1.78,6.5,1.72,5.5,1.64,4.5,1.72,1.5,2.50,NaN,NaN,NaN,NaN,NaN
4,Kentavious Caldwell-Pope,14.5,1.86,12.5,1.84,10.5,1.72,4.5,1.60,3.5,2.20,1.5,1.60,5.75,6.30,7.50,7.75,NaN
5,Kyle Kuzma,25.5,1.86,22.5,1.76,16.5,1.74,10.5,1.82,7.5,1.62,2.5,2.20,3.50,6.00,6.70,7.75,NaN
6,Montrezl Harrell,32.5,1.70,30.5,1.86,21.5,1.82,13.5,1.86,10.5,1.70,2.5,2.15,1.95,3.50,5.90,7.75,NaN
7,Spencer Dinwiddie,27.5,1.76,21.5,1.84,23.5,1.84,11.5,1.84,4.5,1.76,6.5,1.74,2.55,3.80,5.60,7.25,8.00
8,Jaylen Brown,30.5,1.80,27.5,1.76,24.5,1.76,8.5,1.78,5.5,1.70,2.5,1.72,1.47,2.40,3.60,5.30,6.70
9,Jayson Tatum,37.5,1.80,33.5,1.80,30.5,1.80,10.5,1.74,6.5,1.66,3.5,1.70,1.25,1.60,2.20,3.00,4.20


In [391]:
Spe_Joueurs_indx[0]

'PTS_REB_AST'

In [443]:
df2["FullName"] = df2["Player"].apply(lambda p: p.replace("-", " ").replace("'", " ").lower())
df["FullName"] = df["URL"].apply(lambda p: p.split("/")[len(p.split("/"))-1].replace("-", " ").replace("'", " ").lower())

final = df2.merge(df, on=["FullName"], how='left')
final.head()

def Perf(Min, Max, Bet):
    if float(Min) > float(Bet):
        return(float(Min) - float(Bet))
    else:
        if float(Max) < float(Bet):
            return(float(Max) - float(Bet))
        else:
            return(0)

        
final["res_PTS_REB_AST"] = final.apply(lambda p : Perf(p["PTS_REB_AST","min"], p["PTS_REB_AST","max"], p["PTS_REB_AST"]), axis=1)
final["res_PTS_REB"] = final.apply(lambda p : Perf(p["PTS_REB","min"], p["PTS_REB","max"], p["PTS_REB"]), axis=1)
final["res_PTS_AST"] = final.apply(lambda p : Perf(p["PTS_AST","min"], p["PTS_AST","max"], p["PTS_AST"]), axis=1)
final["res_REB_AST"] = final.apply(lambda p : Perf(p["REB_AST","min"], p["REB_AST","max"], p["REB_AST"]), axis=1)
final["res_REB"] = final.apply(lambda p : Perf(p["REB","min"], p["REB","max"], p["REB"]), axis=1)
final["res_AST"] = final.apply(lambda p : Perf(p["AST","min"], p["AST","max"], p["AST"]), axis=1)
final["res"] = final["res_PTS_REB_AST"] + final["res_PTS_REB"] + final["res_PTS_AST"] + final["res_REB_AST"] + final["res_REB"] + final["res_AST"]
final["URL"] = final["URL",""]
final["NbGames"] = final["NbGames",""]
final = final[(final['res']>0.5) | (final['res']<-0.5)]

final[['URL', 'NbGames', 'res_PTS_REB_AST', 'res_PTS_REB', 'res_PTS_AST', 'res_REB_AST', 'res_REB', 'res_AST']].style.format(make_clickable).background_gradient(cmap=cm, subset=['res_PTS_REB_AST', 'res_PTS_REB', 'res_PTS_AST', 'res_REB_AST', 'res_REB', 'res_AST']).set_precision(0)

C:\Users\Valentin\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Valentin\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,URL,NbGames,res_PTS_REB_AST,res_PTS_REB,res_PTS_AST,res_REB_AST,res_REB,res_AST
19,saddiq-bey,4.0,3.5,0.5,0.0,0.5,0.5,0.0
21,franz-wagner,6.0,0.5,0.5,0.5,0.0,0.0,0.0
27,julius-randle,5.0,0.0,0.0,0.0,1.5,0.0,0.0
65,danilo-gallinari,2.0,0.0,0.0,-0.5,0.0,0.0,-0.5
71,donovan-mitchell,4.0,-0.5,0.0,-1.5,-0.5,0.0,0.0
77,rudy-gobert,4.0,2.5,2.5,0.5,0.5,0.5,0.0
80,desmond-bane,5.0,1.5,3.5,2.5,0.0,0.0,0.0
87,kyle-lowry,4.0,-1.5,-3.5,-3.5,0.0,0.0,0.0
99,jakob-poeltl,5.0,0.0,0.0,1.5,0.0,0.0,0.0
104,jaden-mcdaniels,4.0,0.0,0.0,-2.5,0.0,0.0,0.0
